## 1. 라이브러리 로드

In [1]:
import os
import cv2
import json
import imgaug as ia
from tqdm import tqdm # progress bar 시각화 라이브러리
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage

## 2. 경로 설정

In [10]:
input_image_path = "C:\\Users\\rhoms\\businside\\bus_passenger\\Validation\\img"

input_json_path = "data/val.json"
output_image_path = "data/aug_val_img"
output_json_path = "data/json"
output_json_path = os.path.join(output_json_path, "aug_val.json")

# JSON 파일에서 COCO 데이터 불러오기
with open(input_json_path, "r") as f:
    coco_data = json.load(f)

## 3. 이미지, json 파일 잘 로드되었는지 확인하기

In [11]:
# 이미지 정보 가져오기
image_info = coco_data["images"][0]

# 이미지 파일 경로 생성
image_path = os.path.join(input_image_path, image_info["file_name"])
print()
# 이미지 파일 읽어오기
# Image Shape: (1080, 1920, 3) 이렇게 출력나와야 함
if os.path.exists(image_path):
    image = cv2.imread(image_path)
    if image is not None:
        print("Image Shape:", image.shape)
    else:
        print("Failed to read image:", image_path)
else:
    print("Image file not found:", image_path)

# input_image_path 경로 내 첫 번째 이미지 파일명 가져오기
image_files = os.listdir(input_image_path)
if image_files:
    first_image_filename = image_files[0]
    print("첫 번째 이미지 파일명:", first_image_filename)
else:
    print("이미지 파일이 없습니다.")


Image file not found: c:\Users\rhoms\businside\bus_passenger\Validation\img_allInOne\[district]leave_634C_96.jpg
첫 번째 이미지 파일명: [emergency]accident11_087H_1260.jpg


In [12]:
#print("coco data: ", coco_data)

In [13]:
image_info = coco_data["images"][0]

image_path = os.path.join(input_image_path, image_info["file_name"])
image = cv2.imread(image_path)
print(image_info)


{'height': 1080, 'width': 1920, 'id': 1, 'file_name': 'c:\\Users\\rhoms\\businside\\bus_passenger\\Validation\\img_allInOne\\[district]leave_634C_96.jpg'}


In [14]:
print("Image Path:", image_path)
image = cv2.imread(image_path)
print("Image:", image)
# 이미지 3차원으로 나와야 함

Image Path: c:\Users\rhoms\businside\bus_passenger\Validation\img_allInOne\[district]leave_634C_96.jpg
Image: None


In [15]:
# print("Image Width:", image.shape[1])
# print("Image Height:", image.shape[0])

## 4. Augmentation

In [16]:
# 총 7개 aug 기법 적용하기
seq = iaa.Sequential([
    iaa.Fliplr(0.5),  # 좌우 반전
    iaa.GaussianBlur(sigma=(0, 1.0)),  # 가우시안 블러
    iaa.MultiplyBrightness((0.7, 1.3)),  # 밝기 조절
    iaa.ChangeColorspace(from_colorspace="RGB", to_colorspace="HSV"),
    iaa.WithChannels(0, iaa.Add((-30, 30))),  # 색상 조정
    iaa.ChangeColorspace(from_colorspace="HSV", to_colorspace="RGB"),
    iaa.AdditiveGaussianNoise(scale=(0, 0.1 * 255))  # 가우시안 노이즈 추가
])

# 이미 존재하는 annotation id 값들을 기록하는 세트 생성
existing_annotation_ids = set(ann["id"] for ann in coco_data["annotations"])

# 모든 증강 기법 순차적으로 적용
for aug in seq:
    augmented_images = []
    augmented_annotations = []

    image_files = os.listdir(input_image_path)  # 이미지 경로 내의 이미지 파일 목록 가져오기
    for image_filename in tqdm(image_files, desc="Processing images"):  # progress bar 추가
        image_path = os.path.join(input_image_path, image_filename)
        image = cv2.imread(image_path)

        if image is None:
            continue

        image_info = None
        for img_info in coco_data["images"]:
            if img_info["file_name"] == image_filename:
                image_info = img_info
                break

        if image_info is None:
            continue

        annotations = [ann for ann in coco_data["annotations"] if ann["image_id"] == image_info["id"]]

        if not annotations:
            continue

        augmented_annotations_per_aug = []  # 현재 증강 기법에 대한 바운딩 박스 정보 저장 리스트
        for ann in annotations:
            bbox = BoundingBox(x1=ann["bbox"][0], y1=ann["bbox"][1], x2=ann["bbox"][0] + ann["bbox"][2], y2=ann["bbox"][1] + ann["bbox"][3])
            augmented_image, augmented_bbs = aug(image=image, bounding_boxes=[bbox])

            # 증강된 바운딩 박스 정보를 원본 이미지 크기에 맞게 변환
            augmented_bbs = BoundingBox(x1=augmented_bbs[0].x1, y1=augmented_bbs[0].y1, x2=augmented_bbs[0].x2, y2=augmented_bbs[0].y2)
            
            augmented_images.append(augmented_image)
            augmented_annotations_per_aug.append((augmented_bbs, ann))  # 수정된 부분

        augmented_annotations.append(augmented_annotations_per_aug)
    
    for i, augmented_image in enumerate(augmented_images):
        output_image_file = "augmented_{i}.jpg"
        output_image_path_full = os.path.join(output_image_path, output_image_file)
        cv2.imwrite(output_image_path_full, augmented_image)

    print("Starting data augmentation...")
    
    for i, augmented_image in enumerate(tqdm(augmented_images, desc="Augmenting images")):
        if i < len(augmented_annotations):
            image_filename = image_files[i]
            output_image_file = image_filename.replace(".jpg", f"_aug_{i}.jpg")  # 이미지 파일명 수정
            output_image_path_full = os.path.join(output_image_path, output_image_file)
            cv2.imwrite(output_image_path_full, augmented_image)

            for j, (augmented_bbs, annotation) in enumerate(augmented_annotations[i]):  # 수정된 부분
                new_image_info = image_info.copy()
                new_image_info["file_name"] = output_image_file
                new_image_info["id"] = len(coco_data["images"]) + 1

                new_ann = annotation.copy()

                new_ann["bbox"] = [
                    augmented_bbs.x1,  # x1 좌표
                    augmented_bbs.y1,  # y1 좌표
                    augmented_bbs.x2 - augmented_bbs.x1,  # 너비
                    augmented_bbs.y2 - augmented_bbs.y1  # 높이
                ]

                # 중복되지 않은 annotation id 생성
                new_annotation_id = annotation["id"] + len(existing_annotation_ids)
                while new_annotation_id in existing_annotation_ids:
                    new_annotation_id += 1
                existing_annotation_ids.add(new_annotation_id)

                new_ann["id"] = new_annotation_id
                new_ann["image_id"] = new_image_info["id"]

                coco_data["images"].append(new_image_info)
                coco_data["annotations"].append(new_ann)

    print("Data augmentation finished.")
                
# 수정된 COCO 데이터 형식을 JSON 파일로 저장
with open(output_json_path, "w") as f:
    json.dump(coco_data, f, indent=4)

print("finish!")


Processing images: 100%|█████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 17.08it/s]


Starting data augmentation...


Augmenting images: 0it [00:00, ?it/s]


Data augmentation finished.


Processing images: 100%|█████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 20.70it/s]


...

Data augmentation finished.


Processing images: 100%|█████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 21.83it/s]


Starting data augmentation...


Augmenting images: 0it [00:00, ?it/s]


Data augmentation finished.
finish!
